In [ ]:
%cd ..
%reload_ext autoreload
%autoreload 2

In [ ]:
import pyspiel
import numpy as np

from alpha_one.model.model_manager import OpenSpielModelManager, OpenSpielModelConfig
from alpha_one.utils.mcts import initialize_bot
from alpha_one.game.observer import OmniscientObserver
from alpha_one.alg.imperfect_information import AlphaZeroOmniscientMCTSEvaluator, BasicOmniscientMCTSEvaluator
from open_spiel.python.observation import make_observation
from open_spiel.python.algorithms import mcts
from open_spiel.python.algorithms.alpha_zero import model as model_lib

In [ ]:
game_name = 'kuhn_poker'
game = pyspiel.load_game(game_name)

In [ ]:
config = OpenSpielModelConfig(game, 'mlp', 64, 2, 5e-3, 5e-3, omniscient_observer=True)

In [ ]:
model_manager = OpenSpielModelManager(game_name, 'KP')
#model = model_lib.Model.build_model(config.model_type, input_shape=14, output_size=config.game.num_distinct_actions(), 
#                                    nn_width=config.nn_width, nn_depth=config.nn_depth, weight_decay=config.weight_decay, learning_rate=config.learning_rate, path='asdf')

In [ ]:
model = model_manager.get_checkpoint_manager('x').build_model(config)

In [ ]:
mcts_bot = initialize_bot(game, model, 2, 100, omniscient_observer=True)

In [ ]:
state = game.new_initial_state()

In [ ]:
state.legal_actions()

In [ ]:
root = mcts_bot.mcts_search(state)

if state.is_chance_node():
    policy = np.zeros(game.max_chance_outcomes())
else:
    policy = np.zeros(game.num_distinct_actions())
for c in root.children:
    policy[c.action] = c.explore_count
    print(c.action, c.total_reward / c.explore_count)
policy /= policy.sum()

In [ ]:
policy

In [ ]:
action = 0

In [ ]:
state.apply_action(action)

In [ ]:
state.returns()